In [1]:
import findspark
findspark.init('/usr/hdp/current/spark2-client')
findspark.find()

'/usr/hdp/current/spark2-client'

In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("yarn")\
.appName("ch27Regression").getOrCreate()

#.config("spark.jars.packages","databricks:spark-deep-learning:1.2.0-spark2.3-s_2.11,databricks:tensorframes:0.5.0-s_2.11")\
# .config("spark.driver.memory", "1g")\
# .config("spark.executor.memory", "3g")\
# .config("spark.executor.cores", "4")\
# .config("spark.executor.instances", "4")\


#!/usr/bin/env bash

# This file is sourced when running various Spark programs.
# Copy it as spark-env.sh and edit that to configure Spark for your site.

# Options read in YARN client mode
#SPARK_EXECUTOR_INSTANCES="2" #Number of workers to start (Default: 2)
#SPARK_EXECUTOR_CORES="1" #Number of cores for the workers (Default: 1).
#SPARK_EXECUTOR_MEMORY="1G" #Memory per Worker (e.g. 1000M, 2G) (Default: 1G)
#SPARK_DRIVER_MEMORY="512M" #Memory for Master (e.g. 1000M, 2G) (Default: 512 Mb)
#SPARK_YARN_APP_NAME="spark" #The name of your application (Default: Spark)
#SPARK_YARN_QUEUE="default" #The hadoop queue to use for allocation requests (Default: default)
#SPARK_YARN_DIST_FILES="" #Comma separated list of files to be distributed with the job.
#SPARK_YARN_DIST_ARCHIVES="" #Comma separated list of archives to be distributed with the job.

In [3]:
sc = spark.sparkContext

In [4]:
for x in sc._conf.getAll():
    if "/proxy/" in x[1]:
        print(x[1])

http://rm01.itversity.com:19288/proxy/application_1533622723243_18502


In [5]:
data_dir = "/user/kranthidr/dataSets/spark-guide/"
path = data_dir + "regression"

In [6]:
df = spark.read.load(path)

In [7]:
df.show(10,False)

+--------------+-----+
|features      |label|
+--------------+-----+
|[3.0,10.1,3.0]|2.0  |
|[2.0,1.1,1.0] |1.0  |
|[1.0,0.1,-1.0]|0.0  |
|[1.0,0.1,-1.0]|0.0  |
|[2.0,4.1,1.0] |2.0  |
+--------------+-----+



In [8]:
df.printSchema()

root
 |-- features: vector (nullable = true)
 |-- label: double (nullable = true)



In [9]:
# COMMAND ----------
from pyspark.ml.regression import LinearRegression
lr = LinearRegression().setMaxIter(10).setRegParam(0.3).setElasticNetParam(0.8)


# Init docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                          maxIter=100, regParam=0.0, elasticNetParam=0.0, tol=1e-6, fitIntercept=True,                  
#                          standardization=True, solver="auto", weightCol=None, aggregationDepth=2,                  
#                          loss="squaredError", epsilon=1.35)

In [10]:
print lr.explainParams()

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0, current: 0.8)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0, current: 0.3)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the tr

In [11]:
lrModel = lr.fit(df)

In [12]:
# COMMAND ----------

summary = lrModel.summary

In [13]:
summary.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.12805046585610147|
|-0.14468269261572053|
|-0.41903832622420595|
|-0.41903832622420595|
|  0.8547088792080308|
+--------------------+



In [14]:
print summary.totalIterations
print summary.objectiveHistory
print summary.rootMeanSquaredError
print summary.r2

6
[0.5000000000000001, 0.4315295810362787, 0.3132335933881022, 0.31225692666554117, 0.309150608198303, 0.30915058933480255]
0.473084243922
0.720239122691


In [15]:
# COMMAND ----------

from pyspark.ml.regression import GeneralizedLinearRegression

In [16]:
glr = GeneralizedLinearRegression()\
  .setFamily("gaussian")\
  .setLink("identity")\
  .setMaxIter(10)\
  .setRegParam(0.3)\
  .setLinkPredictionCol("linkOut")
    
#     Init docstring: __init__(self, labelCol="label", featuresCol="features", predictionCol="prediction",                  
#                              family="gaussian", link=None, fitIntercept=True, maxIter=25, tol=1e-6,                  
#                              regParam=0.0, weightCol=None, solver="irls", linkPredictionCol=None,                  
#                              variancePower=0.0, linkPower=None, offsetCol=None)

In [17]:
print glr.explainParams()

family: The name of family which is a description of the error distribution to be used in the model. Supported options: gaussian (default), binomial, poisson, gamma and tweedie. (default: gaussian, current: gaussian)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
link: The name of link function which provides the relationship between the linear predictor and the mean of the distribution function. Supported options: identity, log, inverse, logit, probit, cloglog and sqrt. (current: identity)
linkPower: The index in the power link function. Only applicable to the Tweedie family. (undefined)
linkPredictionCol: link prediction (linear predictor) column name (current: linkOut)
maxIter: max number of iterations (>= 0). (default: 25, current: 10)
offsetCol: The offset column name. If this is not set or empty, we treat all instance offsets as 0.0 (undefined)
predictionCol: pred

In [18]:
glrModel = glr.fit(df)

In [19]:
glrModel.coefficients

DenseVector([0.3661, 0.0466, 0.1831])

In [20]:
glrSummary = glrModel.summary

In [21]:
glrSummary.pValues

[0.7169720777072821, 0.7926659441478499, 0.7169720777072832, 0.954892628845124]

In [22]:
# COMMAND ----------
from pyspark.ml.regression import DecisionTreeRegressor
dtr = DecisionTreeRegressor()

# it docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                        maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                        maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10,                  
#                        impurity="variance", seed=None, varianceCol=None)

In [23]:
print dtr.explainParams()

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
maxBins: Max number of bins for discretizing continuous features.  Must be >=2 and >= number of categories for any categorical feature. (default: 32)
maxDepth: Maximum depth of the tree. 

In [24]:
dtrModel = dtr.fit(df)

In [25]:
# COMMAND ----------
from pyspark.ml.regression import RandomForestRegressor
rf =  RandomForestRegressor()

# nit docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                         maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                         maxMemoryInMB=256, cacheNodeIds=False, checkpointInterval=10,                  
#                         impurity="variance", subsamplingRate=1.0, seed=None, numTrees=20,                  
#                         featureSubsetStrategy="auto")

In [26]:
print rf.explainParams()

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featureSubsetStrategy: The number of features to consider for splits at each tree node. Supported options: auto, all, onethird, sqrt, log2, (0.0-1.0], [1-n]. (default: auto)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
maxBins: Max 

In [27]:
rfModel = rf.fit(df)

In [28]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor()

# it docstring: __init__(self, featuresCol="features", labelCol="label", predictionCol="prediction",                  
#                        maxDepth=5, maxBins=32, minInstancesPerNode=1, minInfoGain=0.0,                  
#                        maxMemoryInMB=256, cacheNodeIds=False, subsamplingRate=1.0,                  
#                        checkpointInterval=10, lossType="squared", maxIter=20, stepSize=0.1, seed=None,                  
#                        impurity="variance")

In [29]:
print gbt.explainParams()

cacheNodeIds: If false, the algorithm will pass trees to executors to match instances with nodes. If true, the algorithm will cache node IDs for each instance. Caching can speed up training of deeper trees. Users can set how often should the cache be checkpointed or disable it by setting checkpointInterval. (default: False)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
featuresCol: features column name. (default: features)
impurity: Criterion used for information gain calculation (case-insensitive). Supported options: variance (default: variance)
labelCol: label column name. (default: label)
lossType: Loss function which GBT tries to minimize (case-insensitive). Supported options: squared, absolute (default: squared)
maxBins: Max number of bins for discretizing continuous fea

In [30]:
gbtModel = gbt.fit(df)

In [31]:
# COMMAND ----------
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.regression import GeneralizedLinearRegression
from pyspark.ml import Pipeline
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [32]:
glr = GeneralizedLinearRegression().setFamily("gaussian").setLink("identity")

In [33]:
glr.fit(df).transform(df).show(10,False)

+--------------+-----+-------------------+
|features      |label|prediction         |
+--------------+-----+-------------------+
|[3.0,10.1,3.0]|2.0  |2.294478527607363  |
|[2.0,1.1,1.0] |1.0  |1.1963190184049062 |
|[1.0,0.1,-1.0]|0.0  |0.14723926380368157|
|[1.0,0.1,-1.0]|0.0  |0.14723926380368157|
|[2.0,4.1,1.0] |2.0  |1.2147239263803684 |
+--------------+-----+-------------------+



In [34]:
pipeline = Pipeline().setStages([glr])

In [35]:
pipeline.fit(df).transform(df).show(10,False)

+--------------+-----+-------------------+
|features      |label|prediction         |
+--------------+-----+-------------------+
|[3.0,10.1,3.0]|2.0  |2.294478527607363  |
|[2.0,1.1,1.0] |1.0  |1.1963190184049062 |
|[1.0,0.1,-1.0]|0.0  |0.14723926380368157|
|[1.0,0.1,-1.0]|0.0  |0.14723926380368157|
|[2.0,4.1,1.0] |2.0  |1.2147239263803684 |
+--------------+-----+-------------------+



In [36]:
evaluator = RegressionEvaluator()\
  .setMetricName("rmse")\
  .setPredictionCol("prediction")\
  .setLabelCol("label")

In [37]:
testeval = evaluator.evaluate(pipeline.fit(df).transform(df))

In [38]:
testeval.real

0.3963019236944559

In [39]:
params = ParamGridBuilder().addGrid(glr.regParam, [0.0, 0.5, 1.0]).build()

In [40]:
cv = CrossValidator()\
  .setEstimator(pipeline)\
  .setEvaluator(evaluator)\
  .setEstimatorParamMaps(params)\
  .setNumFolds(2) # should always be 3 or more but this dataset is small

# nit docstring: __init__(self, estimator=None, estimatorParamMaps=None, evaluator=None, 
#                         numFolds=3,                 
#                         seed=None, parallelism=1)

In [41]:
model = cv.fit(df)

In [42]:
model.avgMetrics

[1.013199681974132, 0.8957483797319524, 0.9222783851141156]

In [43]:
model.bestModel.transform(df).show(10,False)

+--------------+-----+------------------+
|features      |label|prediction        |
+--------------+-----+------------------+
|[3.0,10.1,3.0]|2.0  |2.1314519670731205|
|[2.0,1.1,1.0] |1.0  |1.0370413414782234|
|[1.0,0.1,-1.0]|0.0  |0.3254534182849177|
|[1.0,0.1,-1.0]|0.0  |0.3254534182849177|
|[2.0,4.1,1.0] |2.0  |1.1805998548788201|
+--------------+-----+------------------+



In [44]:
# COMMAND ----------
from pyspark.mllib.evaluation import RegressionMetrics

In [45]:
out = model.transform(df)\
  .select("prediction", "label").rdd.map(lambda x: (float(x[0]), float(x[1])))

In [46]:
metrics = RegressionMetrics(out)

/home/kranthidr/.local/lib/python2.7/site-packages/pandas/core/nanops.py:39: UserWarning: The installed version of bottleneck 0.7.0 is not supported in pandas and will be not be used
The minimum supported version is 1.0.0

  ver=ver, min_ver=_MIN_BOTTLENECK_VERSION), UserWarning)


In [47]:
print "MSE: " + str(metrics.meanSquaredError)
print "RMSE: " + str(metrics.rootMeanSquaredError)
print "R-squared: " + str(metrics.r2)
print "MAE: " + str(metrics.meanAbsoluteError)
print "Explained variance: " + str(metrics.explainedVariance)
# COMMAND ----------

MSE: 0.180381626679
RMSE: 0.424713581934
R-squared: 0.774522966651
MAE: 0.327760058048
Explained variance: 0.444839620832
